<a href="https://colab.research.google.com/github/GustavoBD-Dev/ThesisProjectDDRUCNN/blob/main/ThesisProjectDDRUCNN_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deteccion de Retinopatia Diabetica utilizando Redes Neuronales Convolucionales Version 2 (imagenes recortadas de Kaggle)

## Lectura de CSV de set de datos

In [1]:
# Importamos todas las librerias necesarias
import numpy as np
import pandas as pd
import os
import cv2
from google.colab import files # load file

Realizamos la la carga del archivo `trainLabels_cropped.csv` en el cual se encuentran la informacion del dataset de los datos.

In [2]:
# Carga del archivo csv
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving trainLabels_cropped.csv to trainLabels_cropped.csv
User uploaded file "trainLabels_cropped.csv" with length 899292 bytes


## Analisis de datos

Con la ayuda de la libreria **pandas** podemos realizar una lectura del archivo `csv`, aqui se muestra la información en un DataFrame.

In [4]:
# Lectura del archivo csv
data = pd.read_csv('trainLabels_cropped.csv')
# Mostramos encabezado del archivo
data.head()

,Unnamed: 0,Unnamed: 0.1,image,level
0,0,0,10_left,0
1,1,1,10_right,0
2,2,2,13_left,0
3,3,3,13_right,0
4,4,4,15_left,1


Como se muestra en el encabezado del DataFrame, se tienen cuatro columnas, de las cuales nos interesan la columna tres y cuatro, la columna tres contiene el nombre de la imagen y la columna cuatro el nivel de RD que se le ha etiquetado.